In [84]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import requests
import time
from bs4 import BeautifulSoup


from IPython.display import HTML, display
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
from datetime import datetime

from sklearn.model_selection import KFold
import spacy
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelBinarizer
import json
import pickle

import random
from tqdm.notebook import tqdm


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestClassifier
#from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

## Function for fetching html from Web

In [31]:
def html_link_artist(artist):
    '''
    Fetches html link of the artist and saves the html in the folder.
    The input has to be given as a string.
    
    '''
    # Contrsuction of the whole url
    
    url_artist = f"https://www.lyrics.com/artist/{artist}"
    resp_artist =  requests.get(url_artist)
    time.sleep(random.random()/10+.01)
    
    # Write as a html file in the local folder
    with open(f'./{artist}.html', 'w') as file:
        file.write(resp_artist.text)



## function for list of artist's lyrics


In [32]:
def list_artist_link_lyrics(artist):
    '''
    Fetches list of links of songs of the artist in www.lyrics.com and returns a list containing
    link of the lyrics and song names. The input has to be given as a string.
    
    '''
    html_link_artist(artist) 
    # Reading the html file
    HtmlFile_artist = open(f"{artist}.html", 'r', encoding='utf-8')
    source_code_artist = HtmlFile_artist.read()
    
    # Use BeautySoup to find the lyrics links
    # Create a BeautifulSoup object
    
    soup_artist = BeautifulSoup(source_code_artist)
    song_links_artist = soup_artist.find_all('td', class_ = "tal qx")
    return song_links_artist
    

In [33]:
def link_lyrics_artist(artist):
    '''
    Create a DataFrame of links of artist song lyrics.
    
    '''
    
    links_artist = []
    for link_tag in list_artist_link_lyrics(artist):
        link = link_tag.find('a').get('href')
        link_text = link_tag.get_text()
       # print(link, link_text)
        links_artist.append((link, link_text))
        
    df_artist = pd.DataFrame(links_artist, columns=[  'song_link', 'song'])   
    df_artist.drop_duplicates(inplace=True, subset=['song'], ignore_index=True)
    return df_artist

## function for Lyrics collection


In [58]:
def lyrics_artist(artist):
    '''
    Create a DataFrame of lyrics of artist songs.
    '''
    lyrics_artist = []
    df = link_lyrics_artist(artist)
    for link in tqdm(df['song_link']):
        link_url = f" https://www.lyrics.com{link}"
        resp_lyrics = requests.get(link_url)
        time.sleep(random.random()/10+.01)
        soup_lyrics = BeautifulSoup(resp_lyrics.text)
        lyrics_text = soup_lyrics.find('pre', id ='lyric-body-text').get_text()
        #print(lyrics_text, song_name)
        lyrics_artist.append(lyrics_text)
        
    # Add the artist name and remove duplications with lyrics
    df['lyrics']= lyrics_artist
    df['artist_name']= f'{artist}'
    df = df.drop_duplicates(subset=['lyrics'])

    
    return df
    
   

## Load songs for The-Beatles, Pink-Floyd, Justin-Bieber, and Taylor-Swift

In [35]:
link_lyrics_artist('The-Beatles')

,song_link,song
0,/lyric/35984628/The+Beatles/Blackbird,Blackbird
1,/lyric/36392416/The+Beatles/My+Bonnie,My Bonnie
2,/lyric/36255422/The+Beatles/Let+It+Be,Let It Be
3,/lyric/36255456/The+Beatles/Maggie+Mae,Maggie Mae
4,/lyric/36255433/The+Beatles/Old+Brown+Shoe,Old Brown Shoe
...,...,...
1351,/lyric/2541468/The+Beatles/Tomorrow+Never+Know...,Tomorrow Never Knows [Mark I - Unknown Take]
1352,/lyric/2541470/The+Beatles/Strawberry+Fields+F...,"Strawberry Fields Forever [Take 2, 3, 4]"
1353,/lyric/2541471/The+Beatles/Strawberry+Fields+F...,"Strawberry Fields Forever [Take 5, 6, 7]"
1354,/lyric/2541475/The+Beatles/All+You+Need+Is+Lov...,All You Need Is Love [Take 58 - Live Televisio...


In [62]:
df_Beatles= lyrics_artist('The-Beatles')

In [64]:
# Save the data for all as a csv file

filename = 'Lyrics_Beatles.csv'

df_Beatles.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Lyrics_Beatles.csv


In [65]:
df_Floyd = lyrics_artist('Pink-Floyd')

In [67]:
# Save the data for all as a csv file

filename = 'Lyrics_Floyd.csv'

df_Floyd.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Lyrics_Floyd.csv


In [68]:
df_Bieber = lyrics_artist('Justin-Bieber')

In [70]:
# Save the data for all as a csv file

filename = 'Lyrics_Bieber.csv'

df_Bieber.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Lyrics_Bieber.csv


In [73]:
df_Swift = lyrics_artist('Taylor-Swift')

In [75]:
# Save the data for all as a csv file

filename = 'Lyrics_Swift.csv'

df_Swift.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Lyrics_Swift.csv


In [78]:
df_artist = pd.concat([df_Beatles,df_Floyd, df_Bieber, df_Swift], ignore_index=True)
#df_artist

In [79]:
# Save the data for all as a csv file

filename = 'Lyrics_Artist.csv'

df_artist.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Lyrics_Artist.csv


## One can use Spacy verb, adverb, noun, and stop word Parts of Speech (POS) tokens and pushes them into a new dataset

In [80]:
def add_spacy_data(dataset, feature_column):
    '''
    Grabs the verb, adverb, noun, and stop word Parts of Speech (POS) 
    tokens and pushes them into a new dataset. returns an 
    enriched dataset'''
    verbs = []
    nouns = []
    adverbs = []
    corpus = []
    nlp = spacy.load('en_core_web_md')
    ##
    for i in range (0, len(dataset)):
        print("Extracting verbs and topics from record {} of {}".format(i+1, len(dataset)), end = "\r")
        song = dataset.iloc[i][feature_column]
        doc = nlp(song)
        spacy_dataframe = pd.DataFrame()
        for token in doc:
            if token.lemma_ == "-PRON-":
                    lemma = token.text
            else:
                lemma = token.lemma_
            row = {
                "Word": token.text,
                "Lemma": lemma,
                "PoS": token.pos_,
                "Stop Word": token.is_stop
            }
            spacy_dataframe = spacy_dataframe.append(row, ignore_index = True)
        verbs.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "VERB"].values))
        nouns.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "NOUN"].values))
        adverbs.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "ADV"].values))
        corpus_clean = " ".join(spacy_dataframe["Lemma"][spacy_dataframe["Stop Word"] == False].values)
        corpus_clean = re.sub(r'[^A-Za-z0-9]+', ' ', corpus_clean)   
        corpus.append(corpus_clean)
    dataset['Verbs'] = verbs
    dataset['Nouns'] = nouns
    dataset['Adverbs'] = adverbs
    dataset['Corpus'] = corpus
    return dataset

In [81]:
prepped_training_data = add_spacy_data(df_artist, 'lyrics')

In [82]:
prepped_training_data.shape

(996, 8)

In [83]:
# Save the data for all as a csv file

filename = 'Lyrics_Artist_spacy.csv'

prepped_training_data.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Lyrics_Artist_spacy.csv
